In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression

In [2]:
train=pd.read_csv("/kaggle/input/mse-2-ai-201-b-ai-d/train.csv")
test=pd.read_csv("/kaggle/input/mse-2-ai-201-b-ai-d/test.csv")

In [3]:
train.sample(5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4,Class
1653,91962.0,1182.8550,466.2375,NaN,0.8389,342.1837,0.9923,92677.0,0.7643,1.8373,0.8260,NaN,0.0051,0.0028,0.5386,0.9897,Siit_Pistachio
764,65994.0,2394.1021,437.4028,213.1060,0.8733,289.8727,0.8685,NaN,0.6656,2.0525,0.1447,0.6627,0.0066,0.0032,0.4392,0.9014,Kirmizi_Pistachio
663,NaN,1402.4320,460.5004,202.9455,0.8977,296.5899,NaN,75684.0,0.6446,2.2691,0.4414,0.6441,0.0067,0.0029,0.4148,0.9412,Kirmizi_Pistachio
1199,83004.0,1997.5260,463.2641,255.4738,0.8342,325.0907,0.8959,92649.0,NaN,1.8134,0.2614,0.7017,0.0056,0.0031,0.4924,0.8930,Siit_Pistachio
673,81075.0,NaN,493.3022,NaN,0.9000,321.2910,0.9559,84819.0,0.6858,2.2944,0.6936,0.6513,0.0061,0.0027,0.4242,0.9733,Kirmizi_Pistachio


In [4]:
test.head()

,id,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4
0,1,57739,1851.574,438.8182,261.1837,0.8036,271.1376,0.7170,80524,0.5507,1.6801,0.2116,0.6179,0.0076,0.0045,0.3818,0.6414
1,2,83522,2062.064,453.1792,335.8377,0.6714,326.1035,0.7482,111637,0.6197,1.3494,0.2468,0.7196,0.0054,0.0040,0.5178,0.6987
2,3,56531,1982.064,422.6460,231.4328,0.8368,268.2862,0.7620,74183,0.5634,1.8262,0.1808,0.6348,0.0075,0.0041,0.4029,0.7359
3,4,80415,1933.293,472.1519,241.1400,0.8597,319.9806,0.8720,92215,0.6852,1.9580,0.2704,0.6777,0.0059,0.0030,0.4593,0.8993
4,5,72137,1239.403,428.7899,223.9258,0.8528,303.0638,0.9222,78219,0.7338,1.9149,0.5901,0.7068,0.0059,0.0031,0.4996,0.9566


In [5]:
train.isnull().sum()

AREA             155
PERIMETER        265
MAJOR_AXIS       225
MINOR_AXIS       205
ECCENTRICITY     110
EQDIASQ          110
SOLIDITY          95
CONVEX_AREA      233
EXTENT           193
ASPECT_RATIO     243
ROUNDNESS         94
COMPACTNESS      282
SHAPEFACTOR_1    237
SHAPEFACTOR_2    115
SHAPEFACTOR_3    118
SHAPEFACTOR_4    126
Class            136
dtype: int64

In [6]:
test.isnull().sum()

id               0
AREA             0
PERIMETER        0
MAJOR_AXIS       0
MINOR_AXIS       0
ECCENTRICITY     0
EQDIASQ          0
SOLIDITY         0
CONVEX_AREA      0
EXTENT           0
ASPECT_RATIO     0
ROUNDNESS        0
COMPACTNESS      0
SHAPEFACTOR_1    0
SHAPEFACTOR_2    0
SHAPEFACTOR_3    0
SHAPEFACTOR_4    0
dtype: int64

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824 entries, 0 to 1823
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   AREA           1669 non-null   float64
 1   PERIMETER      1559 non-null   float64
 2   MAJOR_AXIS     1599 non-null   float64
 3   MINOR_AXIS     1619 non-null   float64
 4   ECCENTRICITY   1714 non-null   float64
 5   EQDIASQ        1714 non-null   float64
 6   SOLIDITY       1729 non-null   float64
 7   CONVEX_AREA    1591 non-null   float64
 8   EXTENT         1631 non-null   float64
 9   ASPECT_RATIO   1581 non-null   float64
 10  ROUNDNESS      1730 non-null   float64
 11  COMPACTNESS    1542 non-null   float64
 12  SHAPEFACTOR_1  1587 non-null   float64
 13  SHAPEFACTOR_2  1709 non-null   float64
 14  SHAPEFACTOR_3  1706 non-null   float64
 15  SHAPEFACTOR_4  1698 non-null   float64
 16  Class          1688 non-null   object 
dtypes: float64(16), object(1)
memory usage: 242.4+ KB


In [8]:
test.sample(5)

,id,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4
333,334,75511,1895.1591,424.7591,257.4445,0.7954,310.0703,0.8716,86637,0.7243,1.6499,0.2642,0.7300,0.0056,0.0034,0.5329,0.8792
154,155,84292,1238.2000,428.8432,255.9749,0.8023,327.6033,0.9621,87609,0.7599,1.6753,0.6909,0.7639,0.0051,0.0030,0.5836,0.9777
371,372,74213,1158.4590,437.6980,224.3232,0.8587,307.3938,0.9472,78352,0.7550,1.9512,0.6949,0.7023,0.0059,0.0030,0.4932,0.9624
159,160,76722,1600.1870,464.1401,231.3345,0.8669,312.5468,0.8693,88258,0.6772,2.0064,0.3765,0.6734,0.0060,0.0030,0.4535,0.9098
497,498,52067,1049.9200,402.4246,171.1242,0.9051,257.4757,0.9457,55054,0.6628,2.3517,0.5936,0.6398,0.0077,0.0033,0.4094,0.9627


In [9]:
test_id=test['id']
test=test.drop(columns=['id'])

In [10]:
X=train.drop(columns=['Class'])
y=train['Class']

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
numeric_features=X.select_dtypes(include=['int64','float64']).columns
categorical_features=X.select_dtypes(include=['object']).columns

In [13]:
numerical_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])
categorical_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('encode',OneHotEncoder(handle_unknown='ignore'))
])

In [14]:
preprocessing=ColumnTransformer(transformers=[
    ('num',numerical_pipeline,numeric_features),
    ('cat',categorical_pipeline,categorical_features)
])

In [15]:
model = GradientBoostingClassifier(
    n_estimators=920,
    learning_rate=0.02,
    max_depth=4,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=0.7,
    random_state=42
)

In [16]:
pipeline=Pipeline(steps=[
    ('preprocessor',preprocessing),
    ('model',model)
])

In [17]:
not_null_index = y_train.notnull()
X_train = X_train[not_null_index]
y_train = y_train[not_null_index]

In [18]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['AREA', 'PERIMETER', 'MAJOR_AXIS', 'MINOR_AXIS', 'ECCENTRICITY',
       'EQDIASQ', 'SOLIDITY', 'CONVEX_AREA', 'EXTENT', 'ASPECT_RATIO',
       'ROUNDNESS', 'COMPACTNESS', 'SHAPEFACTOR_1', 'SHAPEFACTOR_2',
       'SHAPEFACTOR_3', 'SHAPEFACTOR_4'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index([], dtype='object'))])),
                ('model',
                 GradientBoostingClassifier(learning_rate=0.02, max_depth=4,
                                            n_estimators=920, random_state=42,
                                            subsample=0.7))])

In [19]:
not_null_index = y_test.notnull()
X_test = X_test.loc[not_null_index]
y_test = y_test.loc[not_null_index]

In [20]:
y_pred=pipeline.predict(X_test)

In [21]:
accu=accuracy_score(y_pred,y_test)

In [22]:
print(accu)

0.9127906976744186


In [23]:
y_final=pipeline.predict(test)

In [24]:
prediction=pd.DataFrame({
    'id':test_id,
    'Class':y_final
})

In [25]:
prediction.to_csv("prediction.csv",index=False)

In [26]:
prediction.head()

,id,Class
0,1,Kirmizi_Pistachio
1,2,Siit_Pistachio
2,3,Kirmizi_Pistachio
3,4,Kirmizi_Pistachio
4,5,Kirmizi_Pistachio
